# Entrainement d'un modèle

In [35]:

import pandas as pd

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from SparseMLP import SparseMLP



In [36]:
data_hotel = pd.read_csv("clean_data.csv")
data_hotel

,lead_time,arrival_month,repeated_guest,avg_price_per_room,no_of_special_requests,nb_persons,nb_nights,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Not_Canceled
0,1.609764,0.839446,-0.161398,-1.125010,-0.794241,0.100908,0.008417,-0.060132,-0.100847,-0.242851,1.568506,-1.338085,1.0
1,-0.937707,1.166097,-0.161398,0.117871,0.442749,0.100908,1.190305,-0.060132,-0.100847,-0.242851,-0.637549,0.747336,1.0
2,-0.984236,-1.773767,-0.161398,-1.274108,-0.794241,-1.492329,0.008417,-0.060132,-0.100847,-0.242851,-0.637549,0.747336,0.0
3,1.458545,-0.793812,-0.161398,-0.081324,-0.794241,0.100908,-0.582528,-0.060132,-0.100847,-0.242851,-0.637549,0.747336,0.0
4,-0.437518,-1.120464,-0.161398,-0.245332,-0.794241,0.100908,-0.582528,-0.060132,-0.100847,-0.242851,-0.637549,0.747336,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29694,2.749729,1.492749,-0.161398,-1.512665,-0.794241,0.100908,-0.582528,-0.060132,-0.100847,-0.242851,1.568506,-1.338085,1.0
29695,0.062670,-0.793812,-0.161398,-0.826814,-0.794241,0.100908,0.008417,-0.060132,-0.100847,-0.242851,1.568506,-1.338085,1.0
29696,-0.716693,-1.120464,-0.161398,0.425609,-0.794241,0.100908,-0.582528,-0.060132,-0.100847,-0.242851,-0.637549,0.747336,0.0
29697,2.098321,1.492749,-0.161398,-0.826814,-0.794241,-1.492329,-0.582528,-0.060132,-0.100847,-0.242851,1.568506,-1.338085,1.0


## PyTorch

#### 0. Configuration:

In [37]:
chosen_hidden_dim=1000 # defaut 50
chosen_sparsity=0.8 # defaut 0.8
num_epochs = 5 #defaut 5
batch_size=1000 # defaut 64
###################

test_sparsitees = [0.4,0.5,0.6,0.7,0.8,0.9] # 0.6 a l'air de donner de bon res
test_zeta = [0.3,0.4,0.5,0.6] # 0.5 a l'air mieux

test_hidden_layers = [10,50,100,150,200] #100 a l'air plus opti
test_batch_size = [32,64,128, 256, 512] #32
test_epochs= [50,100, 250, 500] # 500



#### 3. Fonction de mise à jour SET : prune + grow

In [38]:
def set_update(model, zeta=0.3):
    for layer in [model.fc1, model.fc2]:
        weight = layer.weight.data
        mask = layer.mask.data

        # valeurs absolues des poids actifs
        active_weights = weight[mask.bool()].abs()

        # seuil pruning (percentile)
        threshold = torch.quantile(active_weights, zeta)

        # PRUNE : mettre à 0 les connexions trop faibles
        prune_mask = (weight.abs() >= threshold).float()
        new_mask = mask * prune_mask

        # CALCULER NOMBRE DE CONNEXIONS MANQUANTES
        n_removed = int(mask.sum() - new_mask.sum())

        # GROW : ajouter n_removed connexions aléatoirement
        grow_indices = torch.nonzero(new_mask == 0, as_tuple=False)
        perm = torch.randperm(len(grow_indices))[:n_removed]
        new_mask[grow_indices[perm, 0], grow_indices[perm, 1]] = 1.0

        layer.mask.data = new_mask

#### 5. Boucle d’entraînement DST

initialisation

In [39]:

#Metrics:
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
probs_all_list = []
auc_list = []


step = 0
prune_every = 100  # optionnel si tu veux faire du pruning dynamique




# df est ton DataFrame avec 12 features + 1 colonne cible
X = data_hotel.drop(columns=['booking_status_Not_Canceled']).values
y = data_hotel['booking_status_Not_Canceled'].values

# Séparer train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


input_dim = X_train.shape[1]  # 12 features


model = SparseMLP(input_dim=input_dim, hidden_dim=100, output_dim=1, sparsity=0.6, last_activation="elu")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
epoch_nb = 0

for epoch in range(650):
    model.train()
    for batch_x, batch_y in train_loader:
        # Forward
        preds = model(batch_x)
        loss = criterion(preds, batch_y)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Masquer gradients pour les poids déjà masqués
        for layer in [
            model.fc1,
            model.fc2,
            model.fc3,
            model.fc4,
            model.fc5,
        ]:
            layer.weight.grad *= layer.mask

        optimizer.step()
        # SET dynamique
        if step % prune_every == 0 and step > 0:
            set_update(model, zeta=0.5)
        step += 1
    epoch_nb = epoch_nb+1
    # print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")



model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        logits = model(batch_x)
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).float()
        all_preds.append(preds)
        all_labels.append(batch_y)

y_pred = torch.cat(all_preds)
y_true = torch.cat(all_labels)

# Metrices
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
probs_all = torch.sigmoid(model(X_test_tensor)).detach().numpy()
auc = roc_auc_score(y_test, probs_all)


print("accuracy: ", accuracy)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)
print("auc: ", auc)


accuracy:  0.8572390572390572
precision:  0.8881481481481481
recall:  0.9010521042084169
f1:  0.8945535936334246
auc:  0.924252251937139


Affichage des stats

In [40]:
# # Exemple : listes de métriques enregistrées à chaque itération
# # accuracy_list, precision_list, recall_list, f1_list, auc_list
#
# iterations = np.arange(1, len(accuracy_list)+1)
#
# # Créer figure avec 3 lignes x 2 colonnes (le dernier subplot peut rester vide)
# fig, axes = plt.subplots(3, 2, figsize=(12, 12))
# axes = axes.flatten()
#
# # Accuracy
# sns.lineplot(x=iterations, y=auc_list, marker="o", ax=axes[0])
# axes[0].set_title("AUC")
# axes[0].set_xlabel("Iteration")
# axes[0].set_ylabel("AUC")
#
# # Precision
# sns.lineplot(x=iterations, y=precision_list, marker="o", ax=axes[1])
# axes[1].set_title("Precision")
# axes[1].set_xlabel("Iteration")
# axes[1].set_ylabel("Precision")
#
# # Recall
# sns.lineplot(x=iterations, y=recall_list, marker="o", ax=axes[2])
# axes[2].set_title("Recall")
# axes[2].set_xlabel("Iteration")
# axes[2].set_ylabel("Recall")
#
# # F1-score
# sns.lineplot(x=iterations, y=f1_list, marker="o", ax=axes[3])
# axes[3].set_title("F1-score")
# axes[3].set_xlabel("Iteration")
# axes[3].set_ylabel("F1-score")
#
# # AUC
# sns.lineplot(x=iterations, y=accuracy_list, marker="o", ax=axes[4])
# axes[4].set_title("Accuracy")
# axes[4].set_xlabel("Iteration")
# axes[4].set_ylabel("Accuracy")
#
# # Supprimer le dernier subplot vide si nécessaire
# # Dans le dernier subplot (axes[5]), superposer toutes les métriques
# sns.lineplot(x=iterations, y=accuracy_list, marker="o", label="Accuracy", ax=axes[5])
# sns.lineplot(x=iterations, y=precision_list, marker="o", label="Precision", ax=axes[5])
# sns.lineplot(x=iterations, y=recall_list, marker="o", label="Recall", ax=axes[5])
# sns.lineplot(x=iterations, y=f1_list, marker="o", label="F1-score", ax=axes[5])
# sns.lineplot(x=iterations, y=auc_list, marker="o", label="AUC", ax=axes[5])
#
# axes[5].set_title("Toutes les métriques superposées")
# axes[5].set_xlabel("Iteration")
# axes[5].set_ylabel("Valeur")
# axes[5].legend()
#
# plt.tight_layout()
# plt.show()

Res 1:

Parametre:
- hidden_dim= 50
- sparsity = 0.8

Evaluation:
- Accuracy: 0.8160
- Precision: 0.8407
- Recall: 0.8960
- F1-score: 0.8675
- AUC: 0.8685

Res 2

Parametre:
- hidden_dim=50
- sparsity = 0.7

Evaluation:
- Accuracy: 0.8158
- Precision: 0.8357
- Recall: 0.9036
- F1-score: 0.8683
- AUC: 0.8737

Res 3
Parametre:

- hidden_dim: 50
- sparsity: 0.6

Evaluation:
- Accuracy: 0.8177
- Precision: 0.8410
- Recall: 0.8985
- F1-score: 0.8688
- AUC: 0.8760

Res 4

Parametre:

- hidden_dim: 100
- sparsity: 0.8
-
Evaluation:

- Accuracy: 0.8221
- Precision: 0.8468
- Recall: 0.8975
- F1-score: 0.8715
- AUC: 0.8735

## Mesure avec categorie non épurée

- accuracy:  0.8469696969696969
- precision:  0.8564161849710983
- recall:  0.9278557114228457
- f1:  0.8907057833353372
- auc:  0.90887145183591


## Mesure avec categorie epurée

- accuracy:  0.8186868686868687
- precision:  0.8352887048539223
- recall:  0.9095691382765531
- f1:  0.8708478234800335
- auc:  0.8755390658112941

## Mesure avec normalisation standards

- accuracy:  0.8469696969696969
- precision:  0.8564161849710983
- recall:  0.9278557114228457
- f1:  0.8907057833353372
- auc:  0.90887145183591


## Mesure avec normalisation Min Max

- accuracy:  0.8420875420875421
- precision:  0.8596325953838907
- recall:  0.9143286573146293
- f1:  0.8861374119932023
- auc:  0.8983371645755577

## Mesure avec normalisation standards générale

- accuracy:  0.8469696969696969
- precision:  0.8564161849710983
- recall:  0.9278557114228457
- f1:  0.8907057833353372
- auc:  0.90887145183591

## Mesure avec normalisation standards selectionnés

- accuracy:  0.8127946127946128
- precision:  0.8436754176610979
- recall:  0.8855210420841684
- f1:  0.8640919090686874
- auc:  0.8715190905424812

## Mesure avec normalisation min max générale

- accuracy:  0.8420875420875421
- precision:  0.8596325953838907
- recall:  0.9143286573146293
- f1:  0.8861374119932023
- auc:  0.8983371645755577

## Mesure avec normalisation min max selectionnés

- accuracy:  0.812962962962963
- precision:  0.8435487717624612
- recall:  0.8860220440881763
- f1:  0.8642638973732437
- auc:  0.876017242390325

## Mesure avec 3 layers (1 relu, 2 normales)

- accuracy:  0.8469696969696969
- precision:  0.8564161849710983
- recall:  0.9278557114228457
- f1:  0.8907057833353372
- auc:  0.90887145183591

## Mesure avec 5 layers (4 relu, 1 normale)

- accuracy:  0.8636363636363636
- precision:  0.8867282450170151
- recall:  0.9138276553106213
- f1:  0.9000740192450037
- auc:  0.9298901962034952

## Mesure avec 7 layers (6 relu, 1 normale)

- accuracy:  0.8590909090909091
- precision:  0.9031433682596474
- recall:  0.8852705410821643
- f1:  0.8941176470588236
- auc:  0.9265364661561315

## Mesure avec categorie non épurée

- accuracy:  0.8570707070707071
- precision:  0.8835733463509885
- recall:  0.906813627254509
- f1:  0.8950426505130424
- auc:  0.9187879737915255



## Test de différentes layers


## Mesure avec 5 layers (4 relu, 1 normale) - 500 epochs

- accuracy:  0.8636363636363636
- precision:  0.8867282450170151
- recall:  0.9138276553106213
- f1:  0.9000740192450037
- auc:  0.9298901962034952

## Mesure avec 5 layers (4 relu, 1 normale) - 1500 epoch

- accuracy:  0.8582491582491583
- precision:  0.8917910447761194
- recall:  0.8980460921843687
- f1:  0.8949076385421867
- auc:  0.9243697225045573

## Mesure avec 5 layers (4 relu, 1 normale) - 1000 epoch

- accuracy:  0.854040404040404
- precision:  0.8982921233749681
- recall:  0.8827655310621243
- f1:  0.8904611497157296
- auc:  0.9252104697073819

## Mesure avec 5 layers (4 relu, 1 normale) - 750 epoch

- accuracy:  0.8638047138047138
- precision:  0.8920916481892092
- recall:  0.9070641282565131
- f1:  0.8995155881256987
- auc:  0.9300189187409729

## Mesure avec 5 layers (4 relu, 1 normale) - 850 epoch

- accuracy:  0.864983164983165
- precision:  0.8926637124569178
- recall:  0.9083166332665331
- f1:  0.9004221504842315
- auc:  0.9269894383222297


## Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch

- accuracy:  0.8656565656565657
- precision:  0.8976593625498008
- recall:  0.9030561122244489
- f1:  0.9003496503496503
- auc:  0.9323265370576883

## Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - RELU

- accuracy:  0.8656565656565657
- precision:  0.8976593625498008
- recall:  0.9030561122244489
- f1:  0.9003496503496503
- auc:  0.9323265370576883

## Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - leaky relu

- accuracy:  0.8607744107744107
- precision:  0.8858814923189466
- recall:  0.9100701402805611
- f1:  0.8978129247497838
- auc:  0.9230285519704706


## Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - GeLU

- accuracy:  0.8632996632996633
- precision:  0.8889432485322897
- recall:  0.9103206412825652
- f1:  0.8995049504950495
- auc:  0.9249892881245037


## Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - siLu


- accuracy:  0.8585858585858586
- precision:  0.8872235872235872
- recall:  0.904559118236473
- f1:  0.8958074919374845
- auc:  0.9262419860254391



# Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - MiSH

- accuracy:  0.8627946127946128
- precision:  0.8871557397026566
- recall:  0.9118236472945892
- f1:  0.8993205682520075
- auc:  0.9263088548760767



# Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - tanh

- accuracy:  0.8587542087542087
- precision:  0.8823672083434393
- recall:  0.9113226452905812
- f1:  0.896611213801602
- auc:  0.9248135645006647

# Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - elu


- accuracy:  0.8619528619528619
- precision:  0.8788819875776398
- recall:  0.9215931863727455
- f1:  0.8997309855710443
- auc:  0.9267697355697021

# Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - selu

- accuracy:  0.8587542087542087
- precision:  0.8797880028908697
- recall:  0.9148296593186372
- f1:  0.8969667198821073
- auc:  0.9220668621637526

# Test RELU avec couche elu ou mish en couche final

## Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - RELU sans couche final

- accuracy:  0.8656565656565657
- precision:  0.8976593625498008
- recall:  0.9030561122244489
- f1:  0.9003496503496503
- auc:  0.9323265370576883

## Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - RELU Mish en couche final

- accuracy:  0.8641414141414141
- precision:  0.8808897392968189
- recall:  0.9225951903807615
- f1:  0.9012602471552673
- auc:  0.9252056474344994

## Mesure avec 5 layers (4 relu, 1 normale) - 650 epoch - RELU ELU en couche final

- accuracy:  0.8624579124579125
- precision:  0.9013906447534766
- recall:  0.8930360721442886
- f1:  0.8971939096514407
- auc:  0.9251441152325184

# Export du modèle

In [41]:
torch.save({
    "model_state": model.state_dict(),
    "optimizer_state": optimizer.state_dict(),
    "epoch": epoch_nb-1
}, "checkpoint_sparsemlp.pth")
print("Modèle sauvegardé sous modele_sparsemlp.pth")

Modèle sauvegardé sous modele_sparsemlp.pth
